In [1]:
!pip install torch numpy scipy tqdm transformer_lens

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 7.3 MB/s eta 0:00:00
  Created wheel for transformers-stream-generator: filename=transformers_stream_generator-0.0.5-py3-none-any.whl size=12426 sha256=ad8f4c9c0e799a6f9f17986953ae52dd75b861602e7eb2c78dcf8753f256fcff
  Stored in directory: /root/.cache/pip/wheels/a8/58/d2/014cb67c3cc6def738c1b1635dbf4e3dab6fb63aba7070dce0
Successfully built transformers-stream-generator
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: beartype
    Found existing installa

In [5]:
pip install pot


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 73.9 MB/s eta 0:00:00


In [8]:
pip install typing


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26304 sha256=4b9c4a733f9a9ca0dd385d1afcc8d7c165f0ba043f8f51e8ca42835a67ba8f5e
  Stored in directory: /root/.cache/pip/wheels/12/98/52/2bffe242a9a487f00886e43b8ed8dac46456702e11a0d6abef
Successfully built typing


In [4]:
pip install math


ERROR: Could not find a version that satisfies the requirement math (from versions: none)
ERROR: No matching distribution found for math


In [6]:
"""
AKIRA Band Transformer Implementation
=====================================

This implements the core AKIRA architectural innovation: explicit spectral band structure
with differential learning rates.

Key differences from standard transformer:
1. Single MLP replaced with 7 parallel band MLPs
2. Each band has its own learning rate (slow for low-freq, fast for high-freq)
3. Wormhole attention enables cross-band communication
4. Band dimensions follow AKIRA's pattern (larger for low-freq)

AKIRA Project - Experiment 026
Oscar Goldman - Shogu Research Group @ Datamutant.ai
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from typing import List, Tuple, Optional, Dict
from dataclasses import dataclass


@dataclass
class AKIRAConfig:
    """
    Configuration for AKIRA Band Transformer.

    Band structure follows AKIRA's 7-band architecture:
    - Band 0: DC (lowest frequency, most stable features)
    - Band 1-2: Low frequency
    - Band 3: Mid frequency
    - Band 4-5: High frequency
    - Band 6: Highest frequency (most adaptive features)
    """
    # Model dimensions
    vocab_size: int = 50257  # GPT-2 vocab
    num_layers: int = 6
    num_heads: int = 8
    max_seq_length: int = 512
    dropout: float = 0.1

    # Band configuration (7 bands)
    num_bands: int = 7
    band_dims: Tuple[int, ...] = (128, 96, 80, 64, 64, 48, 32)  # Sum = 512

    # Learning rates per band (from slow to fast)
    band_learning_rates: Tuple[float, ...] = (1e-5, 3e-5, 1e-4, 3e-4, 1e-3, 3e-3, 1e-2)

    # MLP expansion factor
    mlp_expansion: int = 4

    # Wormhole attention settings
    use_wormhole: bool = True
    wormhole_threshold: float = 0.5  # Activation threshold for cross-band communication

    @property
    def hidden_dim(self) -> int:
        """Total hidden dimension (sum of all bands)."""
        return sum(self.band_dims)

    def validate(self) -> bool:
        """Validate configuration consistency."""
        assert len(self.band_dims) == self.num_bands, "band_dims must match num_bands"
        assert len(self.band_learning_rates) == self.num_bands, "band_learning_rates must match num_bands"
        assert self.hidden_dim % self.num_heads == 0, "hidden_dim must be divisible by num_heads"
        return True


class BandMLP(nn.Module):
    """
    MLP for a single frequency band.

    Each band has its own MLP with dimension specific to that band.
    Low-frequency bands are larger (more capacity for stable features).
    High-frequency bands are smaller (less capacity, more adaptive).
    """

    def __init__(self, band_dim: int, expansion: int = 4, dropout: float = 0.1):
        super().__init__()

        self.band_dim = band_dim
        hidden_dim = band_dim * expansion

        self.fc1 = nn.Linear(band_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, band_dim)
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.GELU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass through band MLP.

        Args:
            x: Input tensor of shape [batch, seq, band_dim]

        Returns:
            Output tensor of shape [batch, seq, band_dim]
        """
        x = self.fc1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x


class WormholeAttention(nn.Module):
    """
    Wormhole attention for cross-band communication.

    Allows information to "jump" between frequency bands when needed.
    This implements AKIRA's wormhole attention concept: when high-frequency
    patterns need to inform low-frequency representations (or vice versa),
    wormhole attention enables this communication.

    The attention is sparse and gated - it only activates when the
    cross-band relevance exceeds a threshold.

    Uses a COMMON projection dimension for all cross-band queries/keys
    to enable attention between bands of different sizes.
    """

    def __init__(self, band_dims: Tuple[int, ...], threshold: float = 0.5):
        super().__init__()

        self.band_dims = band_dims
        self.num_bands = len(band_dims)
        self.threshold = threshold

        # Common dimension for cross-band attention (use smallest band // 2)
        self.cross_dim = min(band_dims) // 2  # = 16 for default config

        # Projection matrices for cross-band queries and keys
        # All project to the SAME cross_dim for compatibility
        self.cross_queries = nn.ModuleList([
            nn.Linear(dim, self.cross_dim) for dim in band_dims
        ])
        self.cross_keys = nn.ModuleList([
            nn.Linear(dim, self.cross_dim) for dim in band_dims
        ])

        # Values project to target band dimension (for output)
        # We need value projections from each source to each target
        # Simplified: project to common dim, then expand per-target
        self.cross_values = nn.ModuleList([
            nn.Linear(dim, self.cross_dim * 2) for dim in band_dims
        ])

        # Output projection: from common value dim to target band dim
        self.out_projs = nn.ModuleList([
            nn.Linear(self.cross_dim * 2, dim) for dim in band_dims
        ])

        # Gate to control cross-band information flow
        self.gate = nn.ModuleList([
            nn.Linear(dim + dim, dim) for dim in band_dims  # band + projected output
        ])

    def forward(
        self,
        bands: List[torch.Tensor],
        mask: Optional[torch.Tensor] = None
    ) -> List[torch.Tensor]:
        """
        Apply wormhole attention across bands.

        Args:
            bands: List of tensors, one per band, each [batch, seq, band_dim]
            mask: Optional attention mask

        Returns:
            List of updated band tensors
        """
        batch_size, seq_len = bands[0].shape[:2]
        updated_bands = []

        for i, band in enumerate(bands):
            # Compute cross-band attention for this band
            cross_info = torch.zeros(batch_size, seq_len, self.band_dims[i], device=band.device)

            for j, other_band in enumerate(bands):
                if i == j:
                    continue  # Skip self

                # Query from current band, key/value from other band
                # All use common cross_dim for compatibility
                q = self.cross_queries[i](band)  # [batch, seq, cross_dim]
                k = self.cross_keys[j](other_band)  # [batch, seq, cross_dim]
                v = self.cross_values[j](other_band)  # [batch, seq, cross_dim*2]

                # Attention scores
                scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.cross_dim)

                if mask is not None:
                    scores = scores.masked_fill(mask == 0, -1e9)

                attn = F.softmax(scores, dim=-1)

                # Apply attention to values
                cross_output = torch.matmul(attn, v)  # [batch, seq, cross_dim*2]

                # Project to target band dimension
                cross_output = self.out_projs[i](cross_output)  # [batch, seq, band_dim[i]]

                # Apply threshold gate
                gate_input = torch.cat([band, cross_output], dim=-1)
                gate_value = torch.sigmoid(self.gate[i](gate_input))

                # Only allow cross-band info if gate exceeds threshold
                gate_value = (gate_value > self.threshold).float() * gate_value

                cross_info = cross_info + gate_value * cross_output

            # Residual connection with cross-band info
            updated_bands.append(band + cross_info / (self.num_bands - 1))

        return updated_bands


class AKIRAMultiHeadAttention(nn.Module):
    """
    Multi-head attention that operates on concatenated bands.

    Standard attention but aware of band structure for potential
    band-specific attention patterns.
    """

    def __init__(self, config: AKIRAConfig):
        super().__init__()

        self.num_heads = config.num_heads
        self.hidden_dim = config.hidden_dim
        self.head_dim = config.hidden_dim // config.num_heads

        self.q_proj = nn.Linear(config.hidden_dim, config.hidden_dim)
        self.k_proj = nn.Linear(config.hidden_dim, config.hidden_dim)
        self.v_proj = nn.Linear(config.hidden_dim, config.hidden_dim)
        self.out_proj = nn.Linear(config.hidden_dim, config.hidden_dim)

        self.dropout = nn.Dropout(config.dropout)

    def forward(
        self,
        x: torch.Tensor,
        mask: Optional[torch.Tensor] = None
    ) -> torch.Tensor:
        """
        Standard multi-head attention.

        Args:
            x: Input tensor [batch, seq, hidden_dim]
            mask: Optional causal mask

        Returns:
            Output tensor [batch, seq, hidden_dim]
        """
        batch_size, seq_len, _ = x.shape

        # Project to Q, K, V
        q = self.q_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.k_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.v_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)

        # Attention scores
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)

        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)

        # Apply attention to values
        out = torch.matmul(attn, v)
        out = out.transpose(1, 2).contiguous().view(batch_size, seq_len, self.hidden_dim)
        out = self.out_proj(out)

        return out


class AKIRABandLayer(nn.Module):
    """
    Single AKIRA transformer layer with band structure.

    Processing order:
    1. Layer norm
    2. Multi-head attention (on concatenated bands)
    3. Split into bands
    4. Per-band MLP processing
    5. Wormhole attention (cross-band communication)
    6. Concatenate bands
    """

    def __init__(self, config: AKIRAConfig):
        super().__init__()

        self.config = config

        # Pre-norm for attention
        self.norm1 = nn.LayerNorm(config.hidden_dim)

        # Multi-head attention
        self.attention = AKIRAMultiHeadAttention(config)

        # Pre-norm for MLPs
        self.band_norms = nn.ModuleList([
            nn.LayerNorm(dim) for dim in config.band_dims
        ])

        # Per-band MLPs
        self.band_mlps = nn.ModuleList([
            BandMLP(dim, config.mlp_expansion, config.dropout)
            for dim in config.band_dims
        ])

        # Wormhole attention (optional)
        if config.use_wormhole:
            self.wormhole = WormholeAttention(config.band_dims, config.wormhole_threshold)
        else:
            self.wormhole = None

        self.dropout = nn.Dropout(config.dropout)

    def split_bands(self, x: torch.Tensor) -> List[torch.Tensor]:
        """Split concatenated representation into bands."""
        bands = []
        start = 0
        for dim in self.config.band_dims:
            bands.append(x[..., start:start + dim])
            start += dim
        return bands

    def concat_bands(self, bands: List[torch.Tensor]) -> torch.Tensor:
        """Concatenate bands into single representation."""
        return torch.cat(bands, dim=-1)

    def forward(
        self,
        x: torch.Tensor,
        mask: Optional[torch.Tensor] = None
    ) -> torch.Tensor:
        """
        Forward pass through AKIRA layer.

        Args:
            x: Input tensor [batch, seq, hidden_dim]
            mask: Optional causal mask

        Returns:
            Output tensor [batch, seq, hidden_dim]
        """
        # Attention block
        residual = x
        x = self.norm1(x)
        x = self.attention(x, mask)
        x = self.dropout(x)
        x = residual + x

        # Split into bands
        bands = self.split_bands(x)

        # Per-band MLP
        processed_bands = []
        for i, (band, norm, mlp) in enumerate(zip(bands, self.band_norms, self.band_mlps)):
            residual = band
            band = norm(band)
            band = mlp(band)
            band = residual + band
            processed_bands.append(band)

        # Wormhole attention (cross-band communication)
        if self.wormhole is not None:
            processed_bands = self.wormhole(processed_bands, mask)

        # Concatenate bands
        x = self.concat_bands(processed_bands)

        return x


class AKIRABandTransformer(nn.Module):
    """
    Full AKIRA Band Transformer model.

    This is the main model class that implements AKIRA's spectral band architecture.
    """

    def __init__(self, config: AKIRAConfig):
        super().__init__()

        config.validate()
        self.config = config

        # Token and position embeddings
        self.token_embedding = nn.Embedding(config.vocab_size, config.hidden_dim)
        self.position_embedding = nn.Embedding(config.max_seq_length, config.hidden_dim)

        self.dropout = nn.Dropout(config.dropout)

        # AKIRA layers
        self.layers = nn.ModuleList([
            AKIRABandLayer(config) for _ in range(config.num_layers)
        ])

        # Final layer norm
        self.final_norm = nn.LayerNorm(config.hidden_dim)

        # Output projection (tied with token embedding by default)
        self.lm_head = nn.Linear(config.hidden_dim, config.vocab_size, bias=False)

        # Initialize weights
        self.apply(self._init_weights)

    def _init_weights(self, module):
        """Initialize weights following GPT-2 conventions."""
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        elif isinstance(module, nn.LayerNorm):
            torch.nn.init.zeros_(module.bias)
            torch.nn.init.ones_(module.weight)

    def get_band_parameters(self) -> Dict[int, List[nn.Parameter]]:
        """
        Get parameters grouped by band for differential learning rates.

        Returns:
            Dictionary mapping band index to list of parameters
        """
        band_params = {i: [] for i in range(self.config.num_bands)}

        for layer in self.layers:
            for i, mlp in enumerate(layer.band_mlps):
                band_params[i].extend(mlp.parameters())
            for i, norm in enumerate(layer.band_norms):
                band_params[i].extend(norm.parameters())

        return band_params

    def get_optimizer_param_groups(self) -> List[Dict]:
        """
        Get parameter groups with per-band learning rates for optimizer.

        Returns:
            List of param group dicts for optimizer
        """
        band_params = self.get_band_parameters()

        param_groups = []

        # Band-specific parameters with differential LRs
        for band_idx, params in band_params.items():
            param_groups.append({
                'params': params,
                'lr': self.config.band_learning_rates[band_idx],
                'name': f'band_{band_idx}'
            })

        # Non-band parameters (embeddings, attention, final norm) with base LR
        non_band_params = []
        non_band_params.extend(self.token_embedding.parameters())
        non_band_params.extend(self.position_embedding.parameters())
        non_band_params.extend(self.final_norm.parameters())
        non_band_params.extend(self.lm_head.parameters())

        for layer in self.layers:
            non_band_params.extend(layer.norm1.parameters())
            non_band_params.extend(layer.attention.parameters())
            if layer.wormhole is not None:
                non_band_params.extend(layer.wormhole.parameters())

        param_groups.append({
            'params': non_band_params,
            'lr': self.config.band_learning_rates[3],  # Use mid-band LR as base
            'name': 'shared'
        })

        return param_groups

    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None
    ) -> Dict[str, torch.Tensor]:
        """
        Forward pass through AKIRA model.

        Args:
            input_ids: Token IDs [batch, seq]
            attention_mask: Optional attention mask
            labels: Optional labels for loss computation

        Returns:
            Dictionary with 'logits' and optionally 'loss'
        """
        batch_size, seq_len = input_ids.shape
        device = input_ids.device

        # Embeddings
        positions = torch.arange(seq_len, device=device).unsqueeze(0).expand(batch_size, -1)
        x = self.token_embedding(input_ids) + self.position_embedding(positions)
        x = self.dropout(x)

        # Causal mask
        causal_mask = torch.triu(torch.ones(seq_len, seq_len, device=device), diagonal=1).bool()
        causal_mask = ~causal_mask  # Flip for attention

        if attention_mask is not None:
            causal_mask = causal_mask & attention_mask.unsqueeze(1).unsqueeze(2)

        # Process through layers
        for layer in self.layers:
            x = layer(x, causal_mask)

        # Final norm and output
        x = self.final_norm(x)
        logits = self.lm_head(x)

        output = {'logits': logits}

        # Compute loss if labels provided
        if labels is not None:
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            loss = F.cross_entropy(
                shift_logits.view(-1, self.config.vocab_size),
                shift_labels.view(-1),
                ignore_index=-100
            )
            output['loss'] = loss

        return output

    def extract_band_activations(
        self,
        input_ids: torch.Tensor,
        layer_idx: int
    ) -> Dict[int, torch.Tensor]:
        """
        Extract per-band activations for AQ analysis.

        Args:
            input_ids: Token IDs [batch, seq]
            layer_idx: Which layer to extract from

        Returns:
            Dictionary mapping band index to activation tensor
        """
        batch_size, seq_len = input_ids.shape
        device = input_ids.device

        # Forward pass up to target layer
        positions = torch.arange(seq_len, device=device).unsqueeze(0).expand(batch_size, -1)
        x = self.token_embedding(input_ids) + self.position_embedding(positions)

        causal_mask = torch.triu(torch.ones(seq_len, seq_len, device=device), diagonal=1).bool()
        causal_mask = ~causal_mask

        for i, layer in enumerate(self.layers):
            x = layer(x, causal_mask)
            if i == layer_idx:
                break

        # Split into bands
        bands = self.layers[layer_idx].split_bands(x)

        return {i: band.detach() for i, band in enumerate(bands)}


# Utility functions

def create_akira_model(
    num_bands: int = 7,
    hidden_dim: int = 512,
    num_layers: int = 6,
    num_heads: int = 8,
    use_wormhole: bool = True
) -> AKIRABandTransformer:
    """
    Create an AKIRA model with default band structure.

    Args:
        num_bands: Number of frequency bands (default 7)
        hidden_dim: Total hidden dimension (will be distributed across bands)
        num_layers: Number of transformer layers
        num_heads: Number of attention heads
        use_wormhole: Whether to use wormhole attention

    Returns:
        Configured AKIRABandTransformer
    """
    # Distribute dimensions across bands (larger for low-freq)
    if num_bands == 7:
        # Default AKIRA distribution
        band_dims = (128, 96, 80, 64, 64, 48, 32)
        band_lrs = (1e-5, 3e-5, 1e-4, 3e-4, 1e-3, 3e-3, 1e-2)
    else:
        # Uniform distribution for other band counts
        base_dim = hidden_dim // num_bands
        band_dims = tuple([base_dim] * num_bands)
        # Logarithmic LR spread
        band_lrs = tuple([1e-5 * (10 ** (i * 3 / (num_bands - 1))) for i in range(num_bands)])

    config = AKIRAConfig(
        num_layers=num_layers,
        num_heads=num_heads,
        num_bands=num_bands,
        band_dims=band_dims,
        band_learning_rates=band_lrs,
        use_wormhole=use_wormhole
    )

    return AKIRABandTransformer(config)


if __name__ == "__main__":
    # Quick test
    print("Creating AKIRA Band Transformer...")

    config = AKIRAConfig()
    model = AKIRABandTransformer(config)

    print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
    print(f"Hidden dim: {config.hidden_dim}")
    print(f"Band dims: {config.band_dims}")
    print(f"Band LRs: {config.band_learning_rates}")

    # Test forward pass
    batch_size, seq_len = 2, 64
    input_ids = torch.randint(0, config.vocab_size, (batch_size, seq_len))
    labels = torch.randint(0, config.vocab_size, (batch_size, seq_len))

    output = model(input_ids, labels=labels)
    print(f"Output logits shape: {output['logits'].shape}")
    print(f"Loss: {output['loss'].item():.4f}")

    # Test band activation extraction
    band_acts = model.extract_band_activations(input_ids, layer_idx=2)
    for band_idx, acts in band_acts.items():
        print(f"Band {band_idx} activations: {acts.shape}")

    # Test optimizer param groups
    param_groups = model.get_optimizer_param_groups()
    for pg in param_groups:
        print(f"Param group '{pg['name']}': {sum(p.numel() for p in pg['params']):,} params, lr={pg['lr']}")

    print("\nAKIRA Band Transformer test complete!")


Creating AKIRA Band Transformer...
Model parameters: 60,972,672
Hidden dim: 512
Band dims: (128, 96, 80, 64, 64, 48, 32)
Band LRs: (1e-05, 3e-05, 0.0001, 0.0003, 0.001, 0.003, 0.01)
Output logits shape: torch.Size([2, 64, 50257])
Loss: 10.9072
Band 0 activations: torch.Size([2, 64, 128])
Band 1 activations: torch.Size([2, 64, 96])
Band 2 activations: torch.Size([2, 64, 80])
Band 3 activations: torch.Size([2, 64, 64])
Band 4 activations: torch.Size([2, 64, 64])
Band 5 activations: torch.Size([2, 64, 48])
Band 6 activations: torch.Size([2, 64, 32])
Param group 'band_0': 791,808 params, lr=1e-05
Param group 'band_1': 446,400 params, lr=3e-05
Param group 'band_2': 310,560 params, lr=0.0001
Param group 'band_3': 199,296 params, lr=0.0003
Param group 'band_4': 199,296 params, lr=0.001
Param group 'band_5': 112,608 params, lr=0.003
Param group 'band_6': 50,496 params, lr=0.01
Param group 'shared': 58,862,208 params, lr=0.0003

AKIRA Band Transformer test complete!


In [7]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers datasets wandb tqdm

Looking in indexes: https://download.pytorch.org/whl/cu118


In [11]:
"""
AKIRA Band Transformer - COLAB VERSION
======================================

AKIRA Project - Experiment 026
Oscar Goldman - Shogu Research @ Datamutant.ai
"""

# ==============================================================================
# CELL 1: INSTALL DEPENDENCIES
# ==============================================================================

!pip install transformers datasets tqdm

In [12]:

# ==============================================================================
# CELL 2: IMPORTS
# ==============================================================================

import os
import json
import math
import time
from datetime import datetime
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

Device: cuda
GPU: NVIDIA A100-SXM4-40GB
Memory: 42.5 GB


In [13]:

# ==============================================================================
# CELL 3: AKIRA CONFIG
# ==============================================================================

@dataclass
class AKIRAConfig:
    """AKIRA Band Transformer configuration."""
    vocab_size: int = 50257
    num_layers: int = 6
    num_heads: int = 8
    max_seq_length: int = 512
    dropout: float = 0.1
    num_bands: int = 7
    band_dims: Tuple[int, ...] = (128, 96, 80, 64, 64, 48, 32)
    band_learning_rates: Tuple[float, ...] = (1e-5, 3e-5, 1e-4, 3e-4, 1e-3, 3e-3, 1e-2)
    mlp_expansion: int = 4
    use_wormhole: bool = True
    wormhole_threshold: float = 0.5

    @property
    def hidden_dim(self) -> int:
        return sum(self.band_dims)

    def validate(self) -> bool:
        assert len(self.band_dims) == self.num_bands
        assert len(self.band_learning_rates) == self.num_bands
        assert self.hidden_dim % self.num_heads == 0
        return True

print("AKIRAConfig defined")

AKIRAConfig defined


In [14]:

# ==============================================================================
# CELL 4: BAND MLP
# ==============================================================================

class BandMLP(nn.Module):
    """MLP for a single frequency band."""

    def __init__(self, band_dim: int, expansion: int = 4, dropout: float = 0.1):
        super().__init__()
        hidden_dim = band_dim * expansion
        self.fc1 = nn.Linear(band_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, band_dim)
        self.dropout = nn.Dropout(dropout)
        self.activation = nn.GELU()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.fc1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x

print("BandMLP defined")

BandMLP defined


In [15]:
class WormholeAttention(nn.Module):
    """Cross-band communication via gated attention."""

    def __init__(self, band_dims: Tuple[int, ...], threshold: float = 0.5):
        super().__init__()
        self.band_dims = band_dims
        self.num_bands = len(band_dims)
        self.threshold = threshold
        self.cross_dim = min(band_dims) // 2

        self.cross_queries = nn.ModuleList([nn.Linear(dim, self.cross_dim) for dim in band_dims])
        self.cross_keys = nn.ModuleList([nn.Linear(dim, self.cross_dim) for dim in band_dims])
        self.cross_values = nn.ModuleList([nn.Linear(dim, self.cross_dim * 2) for dim in band_dims])
        self.out_projs = nn.ModuleList([nn.Linear(self.cross_dim * 2, dim) for dim in band_dims])
        self.gate = nn.ModuleList([nn.Linear(dim * 2, dim) for dim in band_dims])

    def forward(self, bands: List[torch.Tensor], mask: Optional[torch.Tensor] = None) -> List[torch.Tensor]:
        batch_size, seq_len = bands[0].shape[:2]
        updated_bands = []

        # Handle 4D mask from main attention (squeeze to 3D for our use)
        if mask is not None and mask.dim() == 4:
            mask = mask.squeeze(1)  # [batch, 1, seq, seq] -> [batch, seq, seq]

        for i, band in enumerate(bands):
            cross_info = torch.zeros(batch_size, seq_len, self.band_dims[i], device=band.device, dtype=band.dtype)

            for j, other_band in enumerate(bands):
                if i == j:
                    continue

                q = self.cross_queries[i](band)
                k = self.cross_keys[j](other_band)
                v = self.cross_values[j](other_band)

                scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.cross_dim)
                if mask is not None:
                    scores = scores.masked_fill(mask == 0, torch.finfo(scores.dtype).min)

                attn = F.softmax(scores, dim=-1)
                cross_output = torch.matmul(attn, v)
                cross_output = self.out_projs[i](cross_output)

                gate_input = torch.cat([band, cross_output], dim=-1)
                gate_value = torch.sigmoid(self.gate[i](gate_input))
                gate_value = (gate_value > self.threshold).float() * gate_value

                cross_info = cross_info + gate_value * cross_output

            updated_bands.append(band + cross_info / (self.num_bands - 1))

        return updated_bands

print("WormholeAttention defined")

WormholeAttention defined


In [16]:

# ==============================================================================
# CELL 6: MULTI-HEAD ATTENTION
# ==============================================================================

class AKIRAMultiHeadAttention(nn.Module):
    """Standard multi-head attention."""

    def __init__(self, config: AKIRAConfig):
        super().__init__()
        self.num_heads = config.num_heads
        self.hidden_dim = config.hidden_dim
        self.head_dim = config.hidden_dim // config.num_heads

        self.q_proj = nn.Linear(config.hidden_dim, config.hidden_dim)
        self.k_proj = nn.Linear(config.hidden_dim, config.hidden_dim)
        self.v_proj = nn.Linear(config.hidden_dim, config.hidden_dim)
        self.out_proj = nn.Linear(config.hidden_dim, config.hidden_dim)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x: torch.Tensor, mask: Optional[torch.Tensor] = None) -> torch.Tensor:
        batch_size, seq_len, _ = x.shape

        q = self.q_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.k_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.v_proj(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)

        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, torch.finfo(scores.dtype).min)

        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)

        out = torch.matmul(attn, v)
        out = out.transpose(1, 2).contiguous().view(batch_size, seq_len, self.hidden_dim)
        return self.out_proj(out)

print("AKIRAMultiHeadAttention defined")


AKIRAMultiHeadAttention defined


In [17]:
# ==============================================================================
# CELL 7: AKIRA BAND LAYER
# ==============================================================================

class AKIRABandLayer(nn.Module):
    """Single AKIRA transformer layer with band structure."""

    def __init__(self, config: AKIRAConfig):
        super().__init__()
        self.config = config

        self.norm1 = nn.LayerNorm(config.hidden_dim)
        self.attention = AKIRAMultiHeadAttention(config)
        self.band_norms = nn.ModuleList([nn.LayerNorm(dim) for dim in config.band_dims])
        self.band_mlps = nn.ModuleList([BandMLP(dim, config.mlp_expansion, config.dropout) for dim in config.band_dims])

        if config.use_wormhole:
            self.wormhole = WormholeAttention(config.band_dims, config.wormhole_threshold)
        else:
            self.wormhole = None

        self.dropout = nn.Dropout(config.dropout)

    def split_bands(self, x: torch.Tensor) -> List[torch.Tensor]:
        bands = []
        start = 0
        for dim in self.config.band_dims:
            bands.append(x[..., start:start + dim])
            start += dim
        return bands

    def concat_bands(self, bands: List[torch.Tensor]) -> torch.Tensor:
        return torch.cat(bands, dim=-1)

    def forward(self, x: torch.Tensor, mask: Optional[torch.Tensor] = None) -> torch.Tensor:
        # Attention
        residual = x
        x = self.norm1(x)
        x = self.attention(x, mask)
        x = self.dropout(x)
        x = residual + x

        # Split into bands
        bands = self.split_bands(x)

        # Per-band MLP
        processed_bands = []
        for i, (band, norm, mlp) in enumerate(zip(bands, self.band_norms, self.band_mlps)):
            residual = band
            band = norm(band)
            band = mlp(band)
            band = residual + band
            processed_bands.append(band)

        # Wormhole
        if self.wormhole is not None:
            processed_bands = self.wormhole(processed_bands, mask)

        return self.concat_bands(processed_bands)

print("AKIRABandLayer defined")

AKIRABandLayer defined


In [18]:
# ==============================================================================
# CELL 8: AKIRA BAND TRANSFORMER
# ==============================================================================

class AKIRABandTransformer(nn.Module):
    """Full AKIRA Band Transformer."""

    def __init__(self, config: AKIRAConfig):
        super().__init__()
        config.validate()
        self.config = config

        self.token_embedding = nn.Embedding(config.vocab_size, config.hidden_dim)
        self.position_embedding = nn.Embedding(config.max_seq_length, config.hidden_dim)
        self.dropout = nn.Dropout(config.dropout)

        self.layers = nn.ModuleList([AKIRABandLayer(config) for _ in range(config.num_layers)])
        self.final_norm = nn.LayerNorm(config.hidden_dim)
        self.lm_head = nn.Linear(config.hidden_dim, config.vocab_size, bias=False)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        elif isinstance(module, nn.LayerNorm):
            torch.nn.init.zeros_(module.bias)
            torch.nn.init.ones_(module.weight)

    def get_optimizer_param_groups(self) -> List[Dict]:
        """Get parameter groups with per-band learning rates."""
        band_params = {i: [] for i in range(self.config.num_bands)}

        for layer in self.layers:
            for i, mlp in enumerate(layer.band_mlps):
                band_params[i].extend(mlp.parameters())
            for i, norm in enumerate(layer.band_norms):
                band_params[i].extend(norm.parameters())

        param_groups = []
        for band_idx, params in band_params.items():
            param_groups.append({
                'params': params,
                'lr': self.config.band_learning_rates[band_idx],
                'name': f'band_{band_idx}'
            })

        # Non-band parameters
        non_band_params = []
        non_band_params.extend(self.token_embedding.parameters())
        non_band_params.extend(self.position_embedding.parameters())
        non_band_params.extend(self.final_norm.parameters())
        non_band_params.extend(self.lm_head.parameters())

        for layer in self.layers:
            non_band_params.extend(layer.norm1.parameters())
            non_band_params.extend(layer.attention.parameters())
            if layer.wormhole is not None:
                non_band_params.extend(layer.wormhole.parameters())

        param_groups.append({
            'params': non_band_params,
            'lr': self.config.band_learning_rates[3],
            'name': 'shared'
        })

        return param_groups

    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None
    ) -> Dict[str, torch.Tensor]:
        batch_size, seq_len = input_ids.shape
        device = input_ids.device

        positions = torch.arange(seq_len, device=device).unsqueeze(0).expand(batch_size, -1)
        x = self.token_embedding(input_ids) + self.position_embedding(positions)
        x = self.dropout(x)

        causal_mask = torch.triu(torch.ones(seq_len, seq_len, device=device), diagonal=1).bool()
        causal_mask = ~causal_mask

        if attention_mask is not None:
            causal_mask = causal_mask & attention_mask.unsqueeze(1).unsqueeze(2)

        for layer in self.layers:
            x = layer(x, causal_mask)

        x = self.final_norm(x)
        logits = self.lm_head(x)

        output = {'logits': logits}

        if labels is not None:
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            loss = F.cross_entropy(
                shift_logits.view(-1, self.config.vocab_size),
                shift_labels.view(-1),
                ignore_index=-100
            )
            output['loss'] = loss

        return output

print("AKIRABandTransformer defined")

AKIRABandTransformer defined


In [19]:


# ==============================================================================
# CELL 9: TEST MODEL
# ==============================================================================

print("\n" + "="*60)
print("TESTING AKIRA MODEL")
print("="*60)

config = AKIRAConfig()
model = AKIRABandTransformer(config)

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Hidden dim: {config.hidden_dim}")
print(f"Band dims: {config.band_dims}")

# Test forward pass
batch_size, seq_len = 2, 64
input_ids = torch.randint(0, config.vocab_size, (batch_size, seq_len))
labels = torch.randint(0, config.vocab_size, (batch_size, seq_len))

output = model(input_ids, labels=labels)
print(f"Output shape: {output['logits'].shape}")
print(f"Loss: {output['loss'].item():.4f}")

# Show param groups
param_groups = model.get_optimizer_param_groups()
for pg in param_groups:
    print(f"  {pg['name']}: {sum(p.numel() for p in pg['params']):,} params, lr={pg['lr']}")

print("\nModel test complete!")


TESTING AKIRA MODEL
Model parameters: 60,972,672
Hidden dim: 512
Band dims: (128, 96, 80, 64, 64, 48, 32)
Output shape: torch.Size([2, 64, 50257])
Loss: 10.9278
  band_0: 791,808 params, lr=1e-05
  band_1: 446,400 params, lr=3e-05
  band_2: 310,560 params, lr=0.0001
  band_3: 199,296 params, lr=0.0003
  band_4: 199,296 params, lr=0.001
  band_5: 112,608 params, lr=0.003
  band_6: 50,496 params, lr=0.01
  shared: 58,862,208 params, lr=0.0003

Model test complete!


In [20]:



# ==============================================================================
# CELL 10: DATA LOADING
# ==============================================================================

def load_wikitext103(tokenizer, max_length: int = 512, split: str = "train"):
    """Load WikiText-103 dataset."""
    from datasets import load_dataset

    print(f"Loading WikiText-103 {split}...")
    dataset = load_dataset("wikitext", "wikitext-103-v1", split=split)

    def tokenize_function(examples):
        tokenized = tokenizer(
            examples["text"],
            truncation=True,
            max_length=max_length,
            padding="max_length",
            return_tensors="pt"
        )
        tokenized["labels"] = tokenized["input_ids"].clone()
        return tokenized

    dataset = dataset.filter(lambda x: len(x["text"].strip()) > 0)
    tokenized = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)
    tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

    return tokenized

print("Data loading function defined")

Data loading function defined


In [21]:


# ==============================================================================
# CELL 11: TRAINING LOOP
# ==============================================================================

def train_akira(
    model,
    train_loader,
    eval_loader,
    total_steps=10000,
    eval_every=500,
    log_every=50,
    save_path="./akira_checkpoint"
):
    """Train AKIRA model."""
    model = model.to(device)

    # Optimizer with per-band LRs
    param_groups = model.get_optimizer_param_groups()
    optimizer = AdamW(param_groups, weight_decay=0.01)

    # Scheduler
    scheduler = CosineAnnealingLR(optimizer, T_max=total_steps, eta_min=1e-7)

    # Mixed precision
    scaler = torch.amp.GradScaler('cuda') if device == "cuda" else None

    model.train()
    train_iter = iter(train_loader)
    train_losses = []
    best_eval_loss = float('inf')

    pbar = tqdm(range(total_steps), desc="Training")

    for step in pbar:
        try:
            batch = next(train_iter)
        except StopIteration:
            train_iter = iter(train_loader)
            batch = next(train_iter)

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()

        if scaler:
            with torch.amp.autocast('cuda'):
                output = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = output["loss"]
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
        else:
            output = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = output["loss"]
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        scheduler.step()
        train_losses.append(loss.item())

        if step % log_every == 0:
            avg_loss = sum(train_losses[-log_every:]) / len(train_losses[-log_every:])
            pbar.set_postfix({"loss": f"{avg_loss:.4f}", "lr": f"{scheduler.get_last_lr()[0]:.2e}"})

        if step > 0 and step % eval_every == 0:
            eval_loss = evaluate(model, eval_loader)
            print(f"\nStep {step}: eval_loss = {eval_loss:.4f}, ppl = {math.exp(eval_loss):.2f}")

            if eval_loss < best_eval_loss:
                best_eval_loss = eval_loss
                torch.save(model.state_dict(), f"{save_path}_best.pt")
                print(f"  Saved best model!")

            model.train()

    torch.save(model.state_dict(), f"{save_path}_final.pt")
    return model

def evaluate(model, eval_loader, max_batches=50):
    """Evaluate model."""
    model.eval()
    total_loss = 0
    num_batches = 0

    with torch.no_grad():
        for batch in eval_loader:
            if num_batches >= max_batches:
                break

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            with torch.amp.autocast('cuda') if device == "cuda" else torch.no_grad():
                output = model(input_ids, attention_mask=attention_mask, labels=labels)

            total_loss += output["loss"].item()
            num_batches += 1

    return total_loss / num_batches

print("Training functions defined")

Training functions defined


In [22]:



# ==============================================================================
# CELL 12: RUN TRAINING
# ==============================================================================

# Load tokenizer
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Load data (this takes a few minutes)
train_dataset = load_wikitext103(tokenizer, max_length=256, split="train")
eval_dataset = load_wikitext103(tokenizer, max_length=256, split="validation")

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
eval_loader = DataLoader(eval_dataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)

print(f"Train samples: {len(train_dataset)}")
print(f"Eval samples: {len(eval_dataset)}")

Loading WikiText-103 train...


Map:   0%|          | 0/1165029 [00:00<?, ? examples/s]

Loading WikiText-103 validation...


Map:   0%|          | 0/2461 [00:00<?, ? examples/s]

Train samples: 1165029
Eval samples: 2461


In [28]:
# ==============================================================================
# BASELINE TEST - Run this cell after AKIRA training
# ==============================================================================

import math
from dataclasses import dataclass

@dataclass
class BaselineConfig:
    vocab_size: int = 50257
    num_layers: int = 6
    hidden_dim: int = 512
    num_heads: int = 8
    mlp_expansion: int = 4
    max_seq_length: int = 256
    dropout: float = 0.1
    def validate(self):
        assert self.hidden_dim % self.num_heads == 0
        return True

class BaselineMHA(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_heads = config.num_heads
        self.hidden_dim = config.hidden_dim
        self.head_dim = config.hidden_dim // config.num_heads
        self.q_proj = nn.Linear(config.hidden_dim, config.hidden_dim)
        self.k_proj = nn.Linear(config.hidden_dim, config.hidden_dim)
        self.v_proj = nn.Linear(config.hidden_dim, config.hidden_dim)
        self.out_proj = nn.Linear(config.hidden_dim, config.hidden_dim)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x, mask=None):
        B, S, _ = x.shape
        q = self.q_proj(x).view(B, S, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.k_proj(x).view(B, S, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.v_proj(x).view(B, S, self.num_heads, self.head_dim).transpose(1, 2)
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, torch.finfo(scores.dtype).min)
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)
        out = torch.matmul(attn, v)
        out = out.transpose(1, 2).contiguous().view(B, S, self.hidden_dim)
        return self.out_proj(out)

class BaselineMLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        h = config.hidden_dim * config.mlp_expansion
        self.fc1 = nn.Linear(config.hidden_dim, h)
        self.fc2 = nn.Linear(h, config.hidden_dim)
        self.dropout = nn.Dropout(config.dropout)
        self.act = nn.GELU()
    def forward(self, x):
        return self.dropout(self.fc2(self.dropout(self.act(self.fc1(x)))))

class BaselineLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.norm1 = nn.LayerNorm(config.hidden_dim)
        self.attn = BaselineMHA(config)
        self.norm2 = nn.LayerNorm(config.hidden_dim)
        self.mlp = BaselineMLP(config)
        self.dropout = nn.Dropout(config.dropout)
    def forward(self, x, mask=None):
        x = x + self.dropout(self.attn(self.norm1(x), mask))
        x = x + self.mlp(self.norm2(x))
        return x

class BaselineTransformer(nn.Module):
    def __init__(self, config):
        super().__init__()
        config.validate()
        self.config = config
        self.tok_emb = nn.Embedding(config.vocab_size, config.hidden_dim)
        self.pos_emb = nn.Embedding(config.max_seq_length, config.hidden_dim)
        self.dropout = nn.Dropout(config.dropout)
        self.layers = nn.ModuleList([BaselineLayer(config) for _ in range(config.num_layers)])
        self.norm = nn.LayerNorm(config.hidden_dim)
        self.head = nn.Linear(config.hidden_dim, config.vocab_size, bias=False)
        self.apply(self._init)

    def _init(self, m):
        if isinstance(m, nn.Linear):
            nn.init.normal_(m.weight, std=0.02)
            if m.bias is not None: nn.init.zeros_(m.bias)
        elif isinstance(m, nn.Embedding):
            nn.init.normal_(m.weight, std=0.02)
        elif isinstance(m, nn.LayerNorm):
            nn.init.zeros_(m.bias)
            nn.init.ones_(m.weight)

    def forward(self, input_ids, attention_mask=None, labels=None):
        B, S = input_ids.shape
        pos = torch.arange(S, device=input_ids.device).unsqueeze(0).expand(B, -1)
        x = self.dropout(self.tok_emb(input_ids) + self.pos_emb(pos))
        mask = ~torch.triu(torch.ones(S, S, device=input_ids.device), diagonal=1).bool()
        if attention_mask is not None:
            mask = mask & attention_mask.unsqueeze(1).unsqueeze(2)
        for layer in self.layers:
            x = layer(x, mask)
        logits = self.head(self.norm(x))
        out = {'logits': logits}
        if labels is not None:
            out['loss'] = F.cross_entropy(logits[..., :-1, :].reshape(-1, self.config.vocab_size),
                                          labels[..., 1:].reshape(-1), ignore_index=-100)
        return out

print("="*60)
print("BASELINE TRAINING - Compare with AKIRA")
print("="*60)

baseline_config = BaselineConfig()
baseline_model = BaselineTransformer(baseline_config)
print(f"Baseline params: {sum(p.numel() for p in baseline_model.parameters()):,}")

baseline_model = baseline_model.to(device)
optimizer = AdamW(baseline_model.parameters(), lr=3e-4, weight_decay=0.01)
scheduler = CosineAnnealingLR(optimizer, T_max=5000, eta_min=1e-7)
scaler = torch.amp.GradScaler('cuda')

baseline_model.train()
train_iter = iter(train_loader)
best_loss = float('inf')

for step in tqdm(range(5000), desc="Baseline"):
    try:
        batch = next(train_iter)
    except StopIteration:
        train_iter = iter(train_loader)
        batch = next(train_iter)

    ids = batch["input_ids"].to(device)
    mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)

    optimizer.zero_grad()
    with torch.amp.autocast('cuda'):
        out = baseline_model(ids, attention_mask=mask, labels=labels)
        loss = out["loss"]
    scaler.scale(loss).backward()
    scaler.unscale_(optimizer)
    torch.nn.utils.clip_grad_norm_(baseline_model.parameters(), 1.0)
    scaler.step(optimizer)
    scaler.update()
    scheduler.step()

    if step % 500 == 0 and step > 0:
        baseline_model.eval()
        eval_loss = 0
        with torch.no_grad():
            for i, eb in enumerate(eval_loader):
                if i >= 50: break
                with torch.amp.autocast('cuda'):
                    eo = baseline_model(eb["input_ids"].to(device), eb["attention_mask"].to(device), eb["labels"].to(device))
                eval_loss += eo["loss"].item()
        eval_loss /= 50
        print(f"\nStep {step}: eval_loss={eval_loss:.4f}, ppl={math.exp(eval_loss):.2f}")
        if eval_loss < best_loss:
            best_loss = eval_loss
            print("  New best!")
        baseline_model.train()

print("\n" + "="*60)
print("FINAL COMPARISON")
print("="*60)
print(f"AKIRA best loss:    ~1.99 (ppl ~7.3)")
print(f"Baseline best loss: {best_loss:.4f} (ppl {math.exp(best_loss):.2f})")
print("="*60)
if best_loss > 1.99:
    print("RESULT: AKIRA WINS - Band architecture helps!")
else:
    print("RESULT: Baseline wins or tie - Need investigation")
print("="*60)

BASELINE TRAINING - Compare with AKIRA
Baseline params: 70,509,568


Baseline:  10%|█         | 503/5000 [00:21<10:22,  7.22it/s]


Step 500: eval_loss=2.1135, ppl=8.28
  New best!


Baseline:  20%|██        | 1004/5000 [00:43<07:17,  9.13it/s]


Step 1000: eval_loss=1.9898, ppl=7.31
  New best!


...


Step 4000: eval_loss=1.7308, ppl=5.64
  New best!


Baseline:  90%|█████████ | 4505/5000 [03:14<00:54,  9.07it/s]


Step 4500: eval_loss=1.7230, ppl=5.60
  New best!


Baseline: 100%|██████████| 5000/5000 [03:35<00:00, 23.20it/s]


FINAL COMPARISON
AKIRA best loss:    ~1.99 (ppl ~7.3)
Baseline best loss: 1.7230 (ppl 5.60)
RESULT: Baseline wins or tie - Need investigation


In [29]:
# ==============================================================================
# ABLATION: AKIRA WITHOUT WORMHOLE
# ==============================================================================

print("="*60)
print("ABLATION: AKIRA with bands but NO wormhole attention")
print("="*60)

# Redefine config with wormhole disabled
@dataclass
class AKIRAConfigNoWormhole:
    vocab_size: int = 50257
    num_layers: int = 6
    num_heads: int = 8
    max_seq_length: int = 256
    dropout: float = 0.1
    num_bands: int = 7
    band_dims: Tuple[int, ...] = (128, 96, 80, 64, 64, 48, 32)
    band_learning_rates: Tuple[float, ...] = (1e-5, 3e-5, 1e-4, 3e-4, 1e-3, 3e-3, 1e-2)
    mlp_expansion: int = 4
    use_wormhole: bool = False  # DISABLED
    wormhole_threshold: float = 0.5

    @property
    def hidden_dim(self) -> int:
        return sum(self.band_dims)

    def validate(self) -> bool:
        assert len(self.band_dims) == self.num_bands
        assert len(self.band_learning_rates) == self.num_bands
        assert self.hidden_dim % self.num_heads == 0
        return True

# Create model with NO wormhole
ablation_config = AKIRAConfigNoWormhole()
ablation_model = AKIRABandTransformer(ablation_config)
print(f"AKIRA (no wormhole) params: {sum(p.numel() for p in ablation_model.parameters()):,}")

ablation_model = ablation_model.to(device)
param_groups = ablation_model.get_optimizer_param_groups()
optimizer = AdamW(param_groups, weight_decay=0.01)
scheduler = CosineAnnealingLR(optimizer, T_max=5000, eta_min=1e-7)
scaler = torch.amp.GradScaler('cuda')

ablation_model.train()
train_iter = iter(train_loader)
best_loss = float('inf')

for step in tqdm(range(5000), desc="AKIRA no-wormhole"):
    try:
        batch = next(train_iter)
    except StopIteration:
        train_iter = iter(train_loader)
        batch = next(train_iter)

    ids = batch["input_ids"].to(device)
    mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)

    optimizer.zero_grad()
    with torch.amp.autocast('cuda'):
        out = ablation_model(ids, attention_mask=mask, labels=labels)
        loss = out["loss"]
    scaler.scale(loss).backward()
    scaler.unscale_(optimizer)
    torch.nn.utils.clip_grad_norm_(ablation_model.parameters(), 1.0)
    scaler.step(optimizer)
    scaler.update()
    scheduler.step()

    if step % 500 == 0 and step > 0:
        ablation_model.eval()
        eval_loss = 0
        with torch.no_grad():
            for i, eb in enumerate(eval_loader):
                if i >= 50: break
                with torch.amp.autocast('cuda'):
                    eo = ablation_model(eb["input_ids"].to(device), eb["attention_mask"].to(device), eb["labels"].to(device))
                eval_loss += eo["loss"].item()
        eval_loss /= 50
        print(f"\nStep {step}: eval_loss={eval_loss:.4f}, ppl={math.exp(eval_loss):.2f}")
        if eval_loss < best_loss:
            best_loss = eval_loss
            print("  New best!")
        ablation_model.train()

print("\n" + "="*60)
print("ABLATION RESULTS")
print("="*60)
print(f"Baseline:              1.72 (ppl 5.60)")
print(f"AKIRA + wormhole:      1.99 (ppl 7.30)")
print(f"AKIRA no wormhole:     {best_loss:.4f} (ppl {math.exp(best_loss):.2f})")
print("="*60)
if best_loss < 1.72:
    print("Bands help! Wormhole was the problem.")
elif best_loss < 1.99:
    print("Wormhole hurts, but bands still worse than baseline.")
else:
    print("Bands themselves hurt performance.")
print("="*60)

ABLATION: AKIRA with bands but NO wormhole attention
AKIRA (no wormhole) params: 60,015,616


AKIRA no-wormhole:  10%|█         | 502/5000 [00:41<23:30,  3.19it/s]


Step 500: eval_loss=2.1642, ppl=8.71
  New best!


AKIRA no-wormhole:  20%|██        | 1002/5000 [01:23<21:26,  3.11it/s]


Step 1000: eval_loss=2.0257, ppl=7.58
  New best!


...


Step 4000: eval_loss=1.7778, ppl=5.92
  New best!


AKIRA no-wormhole:  90%|█████████ | 4502/5000 [06:22<02:36,  3.18it/s]


Step 4500: eval_loss=1.7713, ppl=5.88
  New best!


AKIRA no-wormhole: 100%|██████████| 5000/5000 [07:02<00:00, 11.84it/s]


ABLATION RESULTS
Baseline:              1.72 (ppl 5.60)
AKIRA + wormhole:      1.99 (ppl 7.30)
AKIRA no wormhole:     1.7713 (ppl 5.88)
Wormhole hurts, but bands still worse than baseline.


In [30]:
# ==============================================================================
# ABLATION 2: AKIRA bands with UNIFORM learning rate
# ==============================================================================

print("="*60)
print("ABLATION 2: AKIRA bands with UNIFORM LR (no wormhole)")
print("="*60)

@dataclass
class AKIRAConfigUniformLR:
    vocab_size: int = 50257
    num_layers: int = 6
    num_heads: int = 8
    max_seq_length: int = 256
    dropout: float = 0.1
    num_bands: int = 7
    band_dims: Tuple[int, ...] = (128, 96, 80, 64, 64, 48, 32)
    band_learning_rates: Tuple[float, ...] = (3e-4, 3e-4, 3e-4, 3e-4, 3e-4, 3e-4, 3e-4)  # ALL SAME
    mlp_expansion: int = 4
    use_wormhole: bool = False
    wormhole_threshold: float = 0.5

    @property
    def hidden_dim(self) -> int:
        return sum(self.band_dims)

    def validate(self) -> bool:
        assert len(self.band_dims) == self.num_bands
        assert len(self.band_learning_rates) == self.num_bands
        assert self.hidden_dim % self.num_heads == 0
        return True

ablation2_config = AKIRAConfigUniformLR()
ablation2_model = AKIRABandTransformer(ablation2_config)
print(f"AKIRA (uniform LR) params: {sum(p.numel() for p in ablation2_model.parameters()):,}")

ablation2_model = ablation2_model.to(device)
param_groups = ablation2_model.get_optimizer_param_groups()
optimizer = AdamW(param_groups, weight_decay=0.01)
scheduler = CosineAnnealingLR(optimizer, T_max=5000, eta_min=1e-7)
scaler = torch.amp.GradScaler('cuda')

ablation2_model.train()
train_iter = iter(train_loader)
best_loss = float('inf')

for step in tqdm(range(5000), desc="AKIRA uniform-LR"):
    try:
        batch = next(train_iter)
    except StopIteration:
        train_iter = iter(train_loader)
        batch = next(train_iter)

    ids = batch["input_ids"].to(device)
    mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)

    optimizer.zero_grad()
    with torch.amp.autocast('cuda'):
        out = ablation2_model(ids, attention_mask=mask, labels=labels)
        loss = out["loss"]
    scaler.scale(loss).backward()
    scaler.unscale_(optimizer)
    torch.nn.utils.clip_grad_norm_(ablation2_model.parameters(), 1.0)
    scaler.step(optimizer)
    scaler.update()
    scheduler.step()

    if step % 500 == 0 and step > 0:
        ablation2_model.eval()
        eval_loss = 0
        with torch.no_grad():
            for i, eb in enumerate(eval_loader):
                if i >= 50: break
                with torch.amp.autocast('cuda'):
                    eo = ablation2_model(eb["input_ids"].to(device), eb["attention_mask"].to(device), eb["labels"].to(device))
                eval_loss += eo["loss"].item()
        eval_loss /= 50
        print(f"\nStep {step}: eval_loss={eval_loss:.4f}, ppl={math.exp(eval_loss):.2f}")
        if eval_loss < best_loss:
            best_loss = eval_loss
            print("  New best!")
        ablation2_model.train()

print("\n" + "="*60)
print("FULL ABLATION RESULTS")
print("="*60)
print(f"Baseline:                    1.72 (ppl 5.60)")
print(f"AKIRA bands + uniform LR:    {best_loss:.4f} (ppl {math.exp(best_loss):.2f})")
print(f"AKIRA bands + diff LR:       1.77 (ppl 5.88)")
print(f"AKIRA bands + diff LR + wh:  1.99 (ppl 7.30)")
print("="*60)
if best_loss < 1.72:
    print("FINDING: Band structure helps with uniform LR!")
elif best_loss < 1.77:
    print("FINDING: Differential LRs hurt! Uniform is better.")
else:
    print("FINDING: Band structure itself is the issue.")
print("="*60)

ABLATION 2: AKIRA bands with UNIFORM LR (no wormhole)
AKIRA (uniform LR) params: 60,015,616


AKIRA uniform-LR:  10%|█         | 503/5000 [00:52<18:28,  4.06it/s]


Step 500: eval_loss=2.1533, ppl=8.61
  New best!


AKIRA uniform-LR:  20%|██        | 1003/5000 [01:35<16:48,  3.96it/s]


Step 1000: eval_loss=2.0275, ppl=7.60
  New best!


...


Step 4000: eval_loss=1.7832, ppl=5.95
  New best!


AKIRA uniform-LR:  90%|█████████ | 4503/5000 [06:28<02:02,  4.06it/s]


Step 4500: eval_loss=1.7758, ppl=5.90
  New best!


AKIRA uniform-LR: 100%|██████████| 5000/5000 [07:08<00:00, 11.67it/s]


FULL ABLATION RESULTS
Baseline:                    1.72 (ppl 5.60)
AKIRA bands + uniform LR:    1.7758 (ppl 5.90)
AKIRA bands + diff LR:       1.77 (ppl 5.88)
AKIRA bands + diff LR + wh:  1.99 (ppl 7.30)
FINDING: Band structure itself is the issue.


In [33]:
"""
AKIRA Spectral Belief Machine - V2 CORRECT IMPLEMENTATION
==========================================================

This implements the ACTUAL theory from the architecture documents:
1. Learnable bandpass filters for spectral decomposition
2. 7 spectral bands + 1 temporal band = 8 total
3. TEMPORAL wormhole with differential windows per band (Heisenberg)
   - Band 0 sees 128 tokens back, Band 6 sees 4 tokens back
   - Complementary pairs (0-6, 1-5, 2-4) exchange across time scales
4. Differential processing modes per band (Geometric/Hybrid/Reactive)
5. Proper temporal band with causal attention

AKIRA Project - Experiment 026 v2
Oscar Goldman - Shogu Research Group @ Datamutant.ai
"""

# ==============================================================================
# CELL 1: INSTALL
# ==============================================================================

# !pip install transformers datasets tqdm

# ==============================================================================
# CELL 2: IMPORTS
# ==============================================================================

import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")





Device: cuda
GPU: NVIDIA A100-SXM4-40GB


In [34]:
# ==============================================================================
# CELL 3: SPECTRAL BELIEF MACHINE CONFIG
# ==============================================================================

@dataclass
class SpectralConfig:
    """Configuration for Spectral Belief Machine."""
    vocab_size: int = 50257
    num_layers: int = 6
    num_heads: int = 8
    embed_dim: int = 512  # Must be divisible by 8 for 8 bands
    max_seq_length: int = 256
    dropout: float = 0.1

    # 7 spectral bands + 1 temporal = 8
    num_spectral_bands: int = 7
    num_total_bands: int = 8

    # Differential learning rates per band (from theory)
    # Band 0 (DC) slowest -> Band 6 fastest, Band 7 (temporal) medium
    # Tightened to 100x range (was 1000x) for training stability
    band_learning_rates: Tuple[float, ...] = (
        3e-5,   # Band 0: DC - identity, eternal patterns
        5e-5,   # Band 1: Coarse structure
        1e-4,   # Band 2: Medium structure
        3e-4,   # Band 3: Transitions (bridge band)
        5e-4,   # Band 4: Fine structure
        1e-3,   # Band 5: Textures
        3e-3,   # Band 6: Edges, details
        3e-4,   # Band 7: Temporal - medium rate
    )

    # Wormhole config
    wormhole_top_k: int = 8  # Sparse selection
    wormhole_threshold: float = 0.5

    @property
    def dim_per_band(self) -> int:
        return self.embed_dim // self.num_total_bands  # 512/8 = 64

    def validate(self) -> bool:
        assert self.embed_dim % self.num_total_bands == 0, "embed_dim must be divisible by 8"
        assert self.dim_per_band % 8 == 0, "dim_per_band must be divisible by 8 for Tensor Cores"
        return True

print("SpectralConfig defined")
print(f"  embed_dim=512 -> dim_per_band={512//8}")



SpectralConfig defined
  embed_dim=512 -> dim_per_band=64


In [35]:
# ==============================================================================
# CELL 4: SPECTRAL DECOMPOSER (Learnable Bandpass Filters)
# ==============================================================================

class SpectralDecomposer(nn.Module):
    """
    Decomposes input into 7 frequency bands using learnable bandpass-like filters.

    Each band has a characteristic "frequency response" learned via convolutions
    with different kernel sizes (small = high freq, large = low freq).

    This captures the spirit of FFT decomposition in a differentiable way.
    """

    def __init__(self, embed_dim: int, num_bands: int = 7):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_bands = num_bands
        self.dim_per_band = embed_dim // 8  # 64 for embed_dim=512

        # Different "kernel sizes" for different frequency bands
        # Band 0 (DC): largest kernel (captures global structure)
        # Band 6 (high freq): smallest kernel (captures local details)
        kernel_sizes = [15, 11, 9, 7, 5, 3, 1]  # Decreasing for higher bands

        # Bandpass-inspired projections
        # Each band uses: input projection + 1D conv (along embedding) + output projection
        self.input_projs = nn.ModuleList([
            nn.Linear(embed_dim, embed_dim) for _ in range(num_bands)
        ])

        # 1D convolutions along sequence dimension to capture different scales
        self.band_convs = nn.ModuleList([
            nn.Conv1d(embed_dim, embed_dim, kernel_size=k, padding=k//2, groups=embed_dim)
            for k in kernel_sizes
        ])

        # Output projections to band dimension
        self.output_projs = nn.ModuleList([
            nn.Linear(embed_dim, self.dim_per_band) for _ in range(num_bands)
        ])

        # Learnable band weights (for soft frequency selection)
        self.band_gates = nn.ParameterList([
            nn.Parameter(torch.ones(embed_dim) * (0.5 + 0.1 * i))  # Higher bands = more gate
            for i in range(num_bands)
        ])

    def forward(self, x: torch.Tensor) -> Dict[int, torch.Tensor]:
        """
        Decompose input into frequency bands.

        Args:
            x: (batch, seq_len, embed_dim)

        Returns:
            Dict of 7 band tensors, each (batch, seq_len, dim_per_band)
        """
        B, T, D = x.shape
        bands = {}

        for band_idx in range(self.num_bands):
            # Input projection
            h = self.input_projs[band_idx](x)

            # Apply bandpass-like convolution (along sequence dimension)
            # Transpose for conv1d: (B, T, D) -> (B, D, T)
            h_t = h.transpose(1, 2)
            h_conv = self.band_convs[band_idx](h_t)
            h = h_conv.transpose(1, 2)  # Back to (B, T, D)

            # Soft frequency gating
            gate = torch.sigmoid(self.band_gates[band_idx])
            h = h * gate.unsqueeze(0).unsqueeze(0)

            # For low bands: emphasize what remains after high-pass
            # For high bands: emphasize the convolution output directly
            if band_idx < 3:
                # Low freq: residual connection (captures what conv smooths)
                h = x - h + h  # Essentially: original minus high-freq + current

            # Project to band dimension
            bands[band_idx] = self.output_projs[band_idx](h)

        return bands

print("SpectralDecomposer defined (learnable bandpass filters)")


SpectralDecomposer defined (learnable bandpass filters)


In [36]:
# ==============================================================================
# CELL 5: SPECTRAL RECONSTRUCTOR
# ==============================================================================

class SpectralReconstructor(nn.Module):
    """Reconstructs signal from 7 spectral bands."""

    def __init__(self, embed_dim: int, num_bands: int = 7):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_bands = num_bands
        self.dim_per_band = embed_dim // 8  # 64 for embed_dim=512

        # Learnable reconstruction from each band
        self.band_recon = nn.ModuleList([
            nn.Linear(self.dim_per_band, embed_dim) for _ in range(num_bands)
        ])

        # Final mixing layer (7 bands * embed_dim -> embed_dim)
        self.mix = nn.Linear(embed_dim * num_bands, embed_dim)

    def forward(self, bands: Dict[int, torch.Tensor]) -> torch.Tensor:
        """
        Reconstruct full signal from bands.

        Args:
            bands: Dict of 7 band tensors, each (batch, seq, dim_per_band)

        Returns:
            (batch, seq_len, embed_dim)
        """
        # Project each band back to full dimension
        reconstructed = []
        for band_idx in range(self.num_bands):
            if band_idx in bands:
                reconstructed.append(self.band_recon[band_idx](bands[band_idx]))

        # Concatenate and mix
        if reconstructed:
            concat = torch.cat(reconstructed, dim=-1)
            return self.mix(concat)
        else:
            # Fallback
            B, T, _ = list(bands.values())[0].shape
            return torch.zeros(B, T, self.embed_dim, device=list(bands.values())[0].device)

print("SpectralReconstructor defined")



SpectralReconstructor defined


In [37]:
# ==============================================================================
# CELL 6: BAND PROCESSORS (Different modes per band)
# ==============================================================================

class GeometricProcessor(nn.Module):
    """
    Geometric/belief processing for low-frequency bands (0-2).
    Slow, deliberate, structure-preserving.
    """
    def __init__(self, dim: int, dropout: float = 0.1):
        super().__init__()
        # Smaller expansion, more residual
        self.fc1 = nn.Linear(dim, dim * 2)
        self.fc2 = nn.Linear(dim * 2, dim)
        self.norm = nn.LayerNorm(dim)
        self.dropout = nn.Dropout(dropout)
        self.gate = nn.Linear(dim, dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        residual = x
        x = self.norm(x)
        h = F.gelu(self.fc1(x))
        h = self.dropout(h)
        h = self.fc2(h)
        # Gated residual - preserves structure
        gate = torch.sigmoid(self.gate(residual))
        return residual + gate * h


class HybridProcessor(nn.Module):
    """
    Hybrid processing for mid-frequency bands (3-4).
    Balanced between structure and adaptation.
    """
    def __init__(self, dim: int, dropout: float = 0.1):
        super().__init__()
        self.fc1 = nn.Linear(dim, dim * 3)
        self.fc2 = nn.Linear(dim * 3, dim)
        self.norm = nn.LayerNorm(dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        residual = x
        x = self.norm(x)
        x = F.gelu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return residual + x


class ReactiveProcessor(nn.Module):
    """
    Reactive/energy processing for high-frequency bands (5-6).
    Fast, adaptive, detail-sensitive.
    """
    def __init__(self, dim: int, dropout: float = 0.1):
        super().__init__()
        # Larger expansion, more expressive
        self.fc1 = nn.Linear(dim, dim * 4)
        self.fc2 = nn.Linear(dim * 4, dim)
        self.norm = nn.LayerNorm(dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        residual = x
        x = self.norm(x)
        x = F.gelu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return residual + x

print("Band processors defined (Geometric/Hybrid/Reactive)")



Band processors defined (Geometric/Hybrid/Reactive)


In [38]:
# ==============================================================================
# CELL 7: TEMPORAL WORMHOLE ATTENTION
# ==============================================================================

class TemporalWormhole(nn.Module):
    """
    Wormhole attention with DIFFERENTIAL TEMPORAL WINDOWS.

    Key insight from Heisenberg uncertainty:
    - Low freq bands: good frequency resolution, poor time resolution -> see FAR back
    - High freq bands: poor frequency resolution, good time resolution -> see RECENT only

    Complementary pairs:
    - Band 0 (sees t-128) <-> Band 6 (sees t-4)
    - Band 1 (sees t-64)  <-> Band 5 (sees t-8)
    - Band 2 (sees t-32)  <-> Band 4 (sees t-16)
    - Band 3 (bridge) -> queries all with medium window

    The wormhole lets slow patterns inform fast patterns ACROSS TIME.
    """

    def __init__(self, dim: int, max_seq_len: int = 256):
        super().__init__()
        self.dim = dim
        self.max_seq_len = max_seq_len

        # Temporal windows per band (positions to look back)
        # Band 0 sees far, Band 6 sees near
        self.temporal_windows = [128, 64, 32, 16, 16, 8, 4]

        # Complementary pairs: (low_band, high_band)
        self.pairs = [(0, 6), (1, 5), (2, 4)]

        # Per-band temporal attention (simplified: just Q, K, V per band)
        self.q_projs = nn.ModuleList([nn.Linear(dim, dim) for _ in range(7)])
        self.k_projs = nn.ModuleList([nn.Linear(dim, dim) for _ in range(7)])
        self.v_projs = nn.ModuleList([nn.Linear(dim, dim) for _ in range(7)])

        # Cross-band projection for wormhole transfer
        self.cross_projs = nn.ModuleDict({
            f"{low}_{high}": nn.Linear(dim, dim)
            for low, high in self.pairs
        })
        self.cross_projs_rev = nn.ModuleDict({
            f"{low}_{high}": nn.Linear(dim, dim)
            for low, high in self.pairs
        })

        # Output gates per band
        self.gates = nn.ModuleList([nn.Linear(dim * 2, dim) for _ in range(7)])

        # Bridge band (3) projections
        self.bridge_proj = nn.Linear(dim, dim)

        # Precompute causal masks with different windows
        self._precompute_masks(max_seq_len)

    def _precompute_masks(self, max_len: int):
        """Create causal masks with different temporal windows per band."""
        for band_idx, window in enumerate(self.temporal_windows):
            # Causal mask: can only see past positions within window
            mask = torch.ones(max_len, max_len)
            for i in range(max_len):
                # Can see from max(0, i-window) to i (inclusive)
                start = max(0, i - window + 1)
                mask[i, start:i+1] = 0
            # 1 = masked (can't attend), 0 = can attend
            self.register_buffer(f"mask_{band_idx}", mask.bool())

    def forward(self, bands: Dict[int, torch.Tensor]) -> Dict[int, torch.Tensor]:
        """
        Apply temporal wormhole attention.

        Each band attends within its temporal window, then
        complementary pairs exchange information across time scales.
        """
        outputs = {k: v.clone() for k, v in bands.items()}
        B, T, D = bands[0].shape

        # Step 1: Each band does self-attention within its temporal window
        band_contexts = {}
        for band_idx in range(7):
            band = bands[band_idx]
            q = self.q_projs[band_idx](band)
            k = self.k_projs[band_idx](band)
            v = self.v_projs[band_idx](band)

            # Attention with band-specific temporal mask
            scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(D)
            mask = getattr(self, f"mask_{band_idx}")[:T, :T]
            scores = scores.masked_fill(mask.unsqueeze(0), torch.finfo(scores.dtype).min)

            attn = F.softmax(scores, dim=-1)
            context = torch.matmul(attn, v)
            band_contexts[band_idx] = context

        # Step 2: Wormhole transfer between complementary pairs
        for low, high in self.pairs:
            key = f"{low}_{high}"

            # Low band (far past) informs high band (recent)
            # "What patterns from the distant past are relevant now?"
            low_context = band_contexts[low]  # Has info from far back
            high_band = bands[high]           # Current high-freq state

            # Project low context to high band space
            wormhole_to_high = self.cross_projs[key](low_context)

            # High band (recent) informs low band
            # "What recent details should update the slow structure?"
            high_context = band_contexts[high]
            wormhole_to_low = self.cross_projs_rev[key](high_context)

            # Gated updates
            gate_high = torch.sigmoid(self.gates[high](
                torch.cat([high_band, wormhole_to_high], dim=-1)
            ))
            gate_low = torch.sigmoid(self.gates[low](
                torch.cat([bands[low], wormhole_to_low], dim=-1)
            ))

            outputs[high] = outputs[high] + gate_high * wormhole_to_high
            outputs[low] = outputs[low] + gate_low * wormhole_to_low

        # Step 3: Bridge band (3) aggregates from all others
        bridge = bands[3]
        bridge_context = band_contexts[3]

        # Average contexts from all other bands
        other_contexts = torch.stack([
            band_contexts[i] for i in range(7) if i != 3
        ], dim=0).mean(dim=0)

        bridge_info = self.bridge_proj(other_contexts)
        gate_bridge = torch.sigmoid(self.gates[3](
            torch.cat([bridge, bridge_info], dim=-1)
        ))
        outputs[3] = outputs[3] + gate_bridge * bridge_info

        return outputs

print("TemporalWormhole defined (differential temporal windows per band)")




TemporalWormhole defined (differential temporal windows per band)


In [39]:
# ==============================================================================
# CELL 8: TEMPORAL BAND (Band 7 - Causal Attention)
# ==============================================================================

class TemporalBand(nn.Module):
    """
    Band 7: Temporal attention across sequence positions.

    Unlike spectral bands (which process features at one time step),
    this band processes the sequence dimension with CAUSAL masking.

    This is orthogonal to spectral processing (Heisenberg).
    """

    def __init__(self, dim: int, max_seq_len: int, num_heads: int = 4, dropout: float = 0.1):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.head_dim = dim // num_heads

        self.q_proj = nn.Linear(dim, dim)
        self.k_proj = nn.Linear(dim, dim)
        self.v_proj = nn.Linear(dim, dim)
        self.out_proj = nn.Linear(dim, dim)

        self.norm = nn.LayerNorm(dim)
        self.dropout = nn.Dropout(dropout)

        # Causal mask
        self.register_buffer(
            "causal_mask",
            torch.triu(torch.ones(max_seq_len, max_seq_len), diagonal=1).bool()
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Causal temporal attention.

        Args:
            x: (batch, seq_len, dim) - aggregated from spectral bands

        Returns:
            (batch, seq_len, dim) - temporally contextualized
        """
        B, T, D = x.shape
        residual = x
        x = self.norm(x)

        # Multi-head attention
        q = self.q_proj(x).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.k_proj(x).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.v_proj(x).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)

        # Attention over TIME (causal)
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        scores = scores.masked_fill(self.causal_mask[:T, :T].unsqueeze(0).unsqueeze(0),
                                     torch.finfo(scores.dtype).min)

        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)

        out = torch.matmul(attn, v)
        out = out.transpose(1, 2).contiguous().view(B, T, D)
        out = self.out_proj(out)

        return residual + out

print("TemporalBand defined (Band 7 - causal attention over time)")


TemporalBand defined (Band 7 - causal attention over time)


In [40]:
# ==============================================================================
# CELL 9: SPECTRAL ATTENTION (For bands 0-6)
# ==============================================================================

class SpectralAttention(nn.Module):
    """
    Standard attention for spectral bands (non-causal).

    Unlike temporal attention, spectral attention can see all positions
    because frequency content is inherently delocalized in time.
    """

    def __init__(self, dim: int, num_heads: int = 4, dropout: float = 0.1):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.head_dim = dim // num_heads

        self.q_proj = nn.Linear(dim, dim)
        self.k_proj = nn.Linear(dim, dim)
        self.v_proj = nn.Linear(dim, dim)
        self.out_proj = nn.Linear(dim, dim)

        self.norm = nn.LayerNorm(dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor, mask: Optional[torch.Tensor] = None) -> torch.Tensor:
        B, T, D = x.shape
        residual = x
        x = self.norm(x)

        q = self.q_proj(x).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.k_proj(x).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.v_proj(x).view(B, T, self.num_heads, self.head_dim).transpose(1, 2)

        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, torch.finfo(scores.dtype).min)

        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)

        out = torch.matmul(attn, v)
        out = out.transpose(1, 2).contiguous().view(B, T, D)
        out = self.out_proj(out)

        return residual + out

print("SpectralAttention defined (non-causal for frequency bands)")



SpectralAttention defined (non-causal for frequency bands)


In [41]:
# ==============================================================================
# CELL 10: SPECTRAL BELIEF MACHINE LAYER
# ==============================================================================

class SpectralBeliefLayer(nn.Module):
    """
    Complete 7+1 architecture layer.

    7 spectral bands for spatial/feature processing.
    1 temporal band for sequence processing.
    Temporal wormhole attention with differential windows per band.
    """

    def __init__(self, config: SpectralConfig):
        super().__init__()
        self.config = config
        dim = config.dim_per_band

        # Spectral decomposition
        self.decomposer = SpectralDecomposer(config.embed_dim, config.num_spectral_bands)

        # Per-band attention (spectral bands use non-causal)
        self.band_attns = nn.ModuleList([
            SpectralAttention(dim, num_heads=2, dropout=config.dropout)
            for _ in range(7)
        ])

        # Per-band processors (different modes)
        self.band_processors = nn.ModuleList([
            GeometricProcessor(dim, config.dropout),   # Band 0: DC
            GeometricProcessor(dim, config.dropout),   # Band 1: Coarse
            GeometricProcessor(dim, config.dropout),   # Band 2: Medium
            HybridProcessor(dim, config.dropout),      # Band 3: Bridge
            HybridProcessor(dim, config.dropout),      # Band 4: Fine
            ReactiveProcessor(dim, config.dropout),    # Band 5: Texture
            ReactiveProcessor(dim, config.dropout),    # Band 6: Edges
        ])

        # Wormhole attention (temporal + complementary)
        self.wormhole = TemporalWormhole(dim, config.max_seq_length)

        # Temporal band (Band 7)
        self.temporal_band = TemporalBand(dim, config.max_seq_length, num_heads=2, dropout=config.dropout)

        # Aggregator: combine spectral bands for temporal processing
        self.temporal_agg = nn.Linear(config.embed_dim - dim, dim)  # 7 bands -> 1

        # Reconstructor
        self.reconstructor = SpectralReconstructor(config.embed_dim, config.num_spectral_bands)

        # Final output projection (include temporal)
        self.output_proj = nn.Linear(config.embed_dim + dim, config.embed_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Full 7+1 processing.

        Args:
            x: (batch, seq_len, embed_dim)

        Returns:
            (batch, seq_len, embed_dim)
        """
        B, T, D = x.shape

        # 1. Spectral decomposition into 7 bands
        bands = self.decomposer(x)

        # 2. Per-band attention + processing (parallel)
        for i in range(7):
            bands[i] = self.band_attns[i](bands[i])
            bands[i] = self.band_processors[i](bands[i])

        # 3. Wormhole cross-band communication
        bands = self.wormhole(bands)

        # 4. Prepare temporal input (aggregate spectral bands)
        spectral_concat = torch.cat([bands[i] for i in range(7)], dim=-1)
        temporal_input = self.temporal_agg(spectral_concat)

        # 5. Temporal band processing (causal)
        temporal_output = self.temporal_band(temporal_input)

        # 6. Reconstruct from spectral bands
        spectral_output = self.reconstructor(bands)

        # 7. Combine spectral + temporal
        combined = torch.cat([spectral_output, temporal_output], dim=-1)
        output = self.output_proj(combined)

        return output

print("SpectralBeliefLayer defined (full 7+1 architecture)")



SpectralBeliefLayer defined (full 7+1 architecture)


In [42]:
# ==============================================================================
# CELL 11: SPECTRAL BELIEF MACHINE (Full Model)
# ==============================================================================

class SpectralBeliefMachine(nn.Module):
    """
    Complete Spectral Belief Machine.

    Implements the full theory:
    - Learnable bandpass spectral decomposition
    - 7 spectral + 1 temporal bands
    - Temporal wormhole with differential windows (Heisenberg)
    - Differential processing modes (Geometric/Hybrid/Reactive)
    - Complementary cross-band communication
    """

    def __init__(self, config: SpectralConfig):
        super().__init__()
        config.validate()
        self.config = config

        # Embeddings
        self.token_embedding = nn.Embedding(config.vocab_size, config.embed_dim)
        self.position_embedding = nn.Embedding(config.max_seq_length, config.embed_dim)
        self.dropout = nn.Dropout(config.dropout)

        # Spectral Belief Layers
        self.layers = nn.ModuleList([
            SpectralBeliefLayer(config) for _ in range(config.num_layers)
        ])

        # Output
        self.final_norm = nn.LayerNorm(config.embed_dim)
        self.lm_head = nn.Linear(config.embed_dim, config.vocab_size, bias=False)

        # Initialize
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        elif isinstance(module, nn.LayerNorm):
            torch.nn.init.zeros_(module.bias)
            torch.nn.init.ones_(module.weight)

    def get_optimizer_param_groups(self) -> List[Dict]:
        """Get parameter groups with differential learning rates per band."""
        # Group parameters by band
        band_params = {i: [] for i in range(8)}
        other_params = []

        for name, param in self.named_parameters():
            assigned = False
            for band_idx in range(7):
                if f'band_processors.{band_idx}' in name or f'band_attns.{band_idx}' in name:
                    band_params[band_idx].append(param)
                    assigned = True
                    break
            if 'temporal_band' in name:
                band_params[7].append(param)
                assigned = True
            if not assigned:
                other_params.append(param)

        param_groups = []
        for band_idx in range(8):
            if band_params[band_idx]:
                param_groups.append({
                    'params': band_params[band_idx],
                    'lr': self.config.band_learning_rates[band_idx],
                    'name': f'band_{band_idx}'
                })

        if other_params:
            param_groups.append({
                'params': other_params,
                'lr': self.config.band_learning_rates[3],  # Bridge rate
                'name': 'shared'
            })

        return param_groups

    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None
    ) -> Dict[str, torch.Tensor]:
        B, T = input_ids.shape

        # Embeddings
        positions = torch.arange(T, device=input_ids.device).unsqueeze(0).expand(B, -1)
        x = self.token_embedding(input_ids) + self.position_embedding(positions)
        x = self.dropout(x)

        # Process through layers
        for layer in self.layers:
            x = layer(x)

        # Output
        x = self.final_norm(x)
        logits = self.lm_head(x)

        output = {'logits': logits}

        if labels is not None:
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            loss = F.cross_entropy(
                shift_logits.view(-1, self.config.vocab_size),
                shift_labels.view(-1),
                ignore_index=-100
            )
            output['loss'] = loss

        return output

print("SpectralBeliefMachine defined (complete model)")






SpectralBeliefMachine defined (complete model)


In [43]:
# ==============================================================================
# CELL 12: TEST MODEL
# ==============================================================================

print("\n" + "="*60)
print("TESTING SPECTRAL BELIEF MACHINE V2")
print("="*60)

config = SpectralConfig()
model = SpectralBeliefMachine(config)

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Embed dim: {config.embed_dim}")
print(f"Dim per band: {config.dim_per_band}")
print(f"Bands: 7 spectral + 1 temporal = 8")

# Test forward
batch_size, seq_len = 2, 64
input_ids = torch.randint(0, config.vocab_size, (batch_size, seq_len))
labels = torch.randint(0, config.vocab_size, (batch_size, seq_len))

output = model(input_ids, labels=labels)
print(f"Output shape: {output['logits'].shape}")
print(f"Loss: {output['loss'].item():.4f}")

# Show param groups
param_groups = model.get_optimizer_param_groups()
print("\nParameter groups with differential LRs:")
for pg in param_groups:
    print(f"  {pg['name']}: {sum(p.numel() for p in pg['params']):,} params, lr={pg['lr']}")

print("\nSpectral Belief Machine V2 test complete!")


TESTING SPECTRAL BELIEF MACHINE V2
Model parameters: 81,485,440
Embed dim: 512
Dim per band: 64
Bands: 7 spectral + 1 temporal = 8
Output shape: torch.Size([2, 64, 50257])
Loss: 10.8848

Parameter groups with differential LRs:
  band_0: 225,792 params, lr=3e-05
  band_1: 225,792 params, lr=5e-05
  band_2: 225,792 params, lr=0.0001
  band_3: 250,368 params, lr=0.0003
  band_4: 250,368 params, lr=0.0005
  band_5: 299,904 params, lr=0.001
  band_6: 299,904 params, lr=0.003
  band_7: 100,608 params, lr=0.0003
  shared: 79,606,912 params, lr=0.0003

Spectral Belief Machine V2 test complete!


In [44]:
# ==============================================================================
# CELL 13: DATA LOADING
# ==============================================================================

def load_wikitext103(tokenizer, max_length: int = 256, split: str = "train"):
    from datasets import load_dataset
    print(f"Loading WikiText-103 {split}...")
    dataset = load_dataset("wikitext", "wikitext-103-v1", split=split)

    def tokenize_function(examples):
        tokenized = tokenizer(
            examples["text"],
            truncation=True,
            max_length=max_length,
            padding="max_length",
            return_tensors="pt"
        )
        tokenized["labels"] = tokenized["input_ids"].clone()
        return tokenized

    dataset = dataset.filter(lambda x: len(x["text"].strip()) > 0)
    tokenized = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)
    tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    return tokenized

print("Data loading function defined")


Data loading function defined


In [45]:
# ==============================================================================
# CELL 14: TRAINING
# ==============================================================================

def train_spectral(model, train_loader, eval_loader, total_steps=5000, eval_every=500, log_every=50):
    model = model.to(device)

    param_groups = model.get_optimizer_param_groups()
    optimizer = AdamW(param_groups, weight_decay=0.01)
    scheduler = CosineAnnealingLR(optimizer, T_max=total_steps, eta_min=1e-7)
    scaler = torch.amp.GradScaler('cuda') if device == "cuda" else None

    model.train()
    train_iter = iter(train_loader)
    best_loss = float('inf')

    for step in tqdm(range(total_steps), desc="SBM Training"):
        try:
            batch = next(train_iter)
        except StopIteration:
            train_iter = iter(train_loader)
            batch = next(train_iter)

        ids = batch["input_ids"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()

        if scaler:
            with torch.amp.autocast('cuda'):
                out = model(ids, labels=labels)
                loss = out["loss"]
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            scaler.step(optimizer)
            scaler.update()
        else:
            out = model(ids, labels=labels)
            loss = out["loss"]
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

        scheduler.step()

        if step % 500 == 0 and step > 0:
            model.eval()
            eval_loss = 0
            with torch.no_grad():
                for i, eb in enumerate(eval_loader):
                    if i >= 50: break
                    with torch.amp.autocast('cuda') if device == "cuda" else torch.no_grad():
                        eo = model(eb["input_ids"].to(device), labels=eb["labels"].to(device))
                    eval_loss += eo["loss"].item()
            eval_loss /= 50
            print(f"\nStep {step}: eval_loss={eval_loss:.4f}, ppl={math.exp(eval_loss):.2f}")
            if eval_loss < best_loss:
                best_loss = eval_loss
                print("  New best!")
            model.train()

    return best_loss

print("Training function defined")




Training function defined


In [46]:
# ==============================================================================
# CELL 15: RUN TRAINING
# ==============================================================================

from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

train_dataset = load_wikitext103(tokenizer, max_length=256, split="train")
eval_dataset = load_wikitext103(tokenizer, max_length=256, split="validation")

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
eval_loader = DataLoader(eval_dataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)

print(f"Train: {len(train_dataset)}, Eval: {len(eval_dataset)}")


Loading WikiText-103 train...


Map:   0%|          | 0/1165029 [00:00<?, ? examples/s]

Loading WikiText-103 validation...


Map:   0%|          | 0/2461 [00:00<?, ? examples/s]

Train: 1165029, Eval: 2461
